In [ ]:
from pyplaces import overture_maps as om
import folium

In [ ]:
import pandas
import geopandas
def plot_geodataframes(gdfs, zoom_start=11, popup_columns=None):
    if not isinstance(gdfs, list):
        gdfs = [gdfs]
    if popup_columns is None:
        popup_columns = [None] * len(gdfs)

    # Filter out empty GeoDataFrames
    gdfs_filtered = [(gdf, popup_columns[i]) for i, gdf in enumerate(gdfs) if not gdf.empty]

    if not gdfs_filtered:
        print("All GeoDataFrames are empty.")
        return None

    # Center the map at the centroid of the first non-empty GeoDataFrame
    all_geoms = gpd.GeoSeries(pd.concat([gdf.geometry for gdf, _ in gdfs_filtered], ignore_index=True))
    center = all_geoms.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=zoom_start, tiles="CartoDB positron")

    for gdf, popup_col in gdfs_filtered:
        # Check if all geometries are Points
        if gdf.geometry.geom_type.isin(['Point']).all():
            for _, row in gdf.iterrows():
                coords = [row.geometry.y, row.geometry.x]
                popup = str(row[popup_col]) if popup_col and popup_col in row else None
                folium.CircleMarker(
                    location=coords,
                    radius=4,  # size of the dot
                    color="orange",
                    fill=True,
                    fill_color="orange",
                    fill_opacity=1,
                    popup=popup
                ).add_to(m)
        else:
            # Otherwise treat as polygon or line geometries
            if popup_col and popup_col in gdf.columns:
                folium.GeoJson(
                    gdf,
                    tooltip=folium.GeoJsonTooltip(fields=[popup_col], aliases=['Name:']),
                    popup=folium.GeoJsonPopup(fields=[popup_col])
                ).add_to(m)
            else:
                folium.GeoJson(gdf).add_to(m)

    return m


In [ ]:
# Get bagel shops
nyc_bagel_gdf = om.overture_places_from_place("New York City", filters="categories.primary = 'bagel_shop'")
nyc_bagel_gdf["primary_name"] = nyc_bagel_gdf["names"].apply(lambda x: x["primary"])

In [ ]:
# Plot bagel shops
m = plot_geodataframes(nyc_bagel_gdf[["id", "primary_name", "geometry"]], popup_columns=["primary_name"])
m